In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import sys
import os

sys.path.append("../../../")
from src.athena import Athena
from src.utils import create_session

## Global

In [3]:
boto3_session = create_session(
    profile_name="dev",
    role_arn="arn:aws:iam::722696965592:role/athena-full-access-role",
)

wait = True
ctas_approach = False

table = "daily_btc"
database = "trading"
sql_path = "../sql/"

athena = Athena(boto3_session=boto3_session, s3_output=os.getenv("ATHENA_S3_OUTPUT"))
athena

Athena(boto3_session=Session(region_name='us-east-1'), s3_output=s3://sql-case-studies/query_results)

## Create Database & Table

In [4]:
response = athena.create_database(database=database, wait=wait)

response

Query executed successfully


In [5]:
with open(os.path.join(sql_path, f"{table}_ddl.sql")) as f:
    ddl_daily_btc = f.read()

print(ddl_daily_btc)

CREATE EXTERNAL TABLE IF NOT EXISTS trading.daily_btc (
  market_date TIMESTAMP,
  open_price DOUBLE,
  high_price DOUBLE,
  low_price DOUBLE,
  close_price DOUBLE,
  adjusted_close_price DOUBLE,
  volume DOUBLE
)
COMMENT 'Daily Bitcoin trading data'
STORED AS PARQUET
LOCATION 's3://sql-case-studies/trading/daily_btc/'
TBLPROPERTIES ('classification'='parquet', 'parquet.compress'='SNAPPY');



In [6]:
response = athena.create_table(database=database, query=ddl_daily_btc, wait=wait)

response

Query executed successfully


In [7]:
athena.query(
    database=database,
    query=f"SELECT * FROM {database}.{table} LIMIT 10;",
    ctas_approach=ctas_approach,
)

,market_date,open_price,high_price,low_price,close_price,adjusted_close_price,volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800.0
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200.0
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700.0
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600.0
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100.0
5,2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600.0
6,2014-09-23,402.092010,441.557007,396.196991,435.790985,435.790985,45099500.0
7,2014-09-24,435.751007,436.112000,421.131989,423.204987,423.204987,30627700.0
8,2014-09-25,423.156006,423.519989,409.467987,411.574005,411.574005,26814400.0
9,2014-09-26,411.428986,414.937988,400.009003,404.424988,404.424988,21460800.0


## Drop Database & Table

In [8]:
athena.drop_table(database=database, table=table, wait=wait)

Query executed successfully


In [9]:
athena.drop_database(database=database, wait=wait)

Query executed successfully
